#### The following are the steps that follow after we have conducted the optimization for the worst case scenarios. 
(According to assignment 9 we should try to reudce the number of policies to at most 50 and then we should reevaluate the set of solutons over 1000 scenarios using LHS)

In [1]:
import pandas as pd

In [ ]:
#results_0 = pd.read_csv('intermediate outputs/multi scenario optimization results/0 optimization results.csv')
#results_1 = pd.read_csv('intermediate outputs/multi scenario optimization results/1 optimization results.csv')
#results_2 = pd.read_csv('intermediate outputs/multi scenario optimization results/2 optimization results.csv')

In [ ]:
#results_2

In [37]:
# putting the results of the worst case scenarios' optimization in a list
results = []
for i in range(3):
    read_results= pd.read_csv("intermediate outputs/multi scenario optimization results/" + str(i) + " optimization results.csv")
    #print(len(read_results))
    results.append(read_results)
    

In [3]:
results

[      Unnamed: 0  0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  1_RfR 2  \
 0              0        1        1        1        0        1        1   
 1              1        1        1        1        1        1        1   
 2              2        1        1        0        1        0        1   
 3              3        1        1        0        1        1        1   
 4              4        0        1        1        0        0        0   
 ...          ...      ...      ...      ...      ...      ...      ...   
 3992        3992        1        0        1        0        0        1   
 3993        3993        0        1        1        1        1        0   
 3994        3994        0        1        1        0        1        0   
 3995        3995        1        1        1        1        1        0   
 3996        3996        0        1        0        1        0        1   
 
       2_RfR 0  2_RfR 1  2_RfR 2  ...  A.4_Dike Investment Costs  \
 0           0        0       

 ### Dont forget, we should try to reduce the number of policies!! So far all solutions have been included.

In [24]:
from ema_workbench import Policy
from problem_formulation import get_model_for_problem_formulation

# create the dike_model
problem_formulation = 6 # WARNING: use the same PF as the ones that you used to create the results csv file!
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)


policies = []
to_drop = []

for i, result in enumerate(results):
    
    for column in results[i].columns:  #we keep only the levers 
        to_drop.append(column)
    for o in dike_model.levers:
        to_drop.remove(o.name)

    result = result.drop([column for column in to_drop], axis=1);
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)

In [35]:
len(policies)

8240

In [ ]:
#run the experiments 
start_time = time.time()
print('Runtime started')
      
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments,outcomes = evaluator.perform_experiments(1000, policies=policies)

end_time = time.time()
print('Runtime ended with duration of', str(end_time - start_time))


#save the results
experiments.to_csv("intermediate outputs/second evaluation - experiments description.csv")

# save the outcomes to a pickle file (outcomes is a dictionary)
a_file = open("intermediate outputs/second evaluation - outcomes description.pkl", "wb")
pickle.dump(outcomes, a_file)
a_file.close()
# to read back our dictionary: https://www.adamsmith.haus/python/answers/how-to-save-a-dictionary-to-a-file-in-python

In [39]:
#calculating the maximum regret 

overall_regret = {}
max_regret = {}

for scenario in experiments.scenario.unique():
    logical = experiments.scenario==scenario
    temp_results = {k:v[logical] for k,v in outcomes.items()}
    temp_results = pd.DataFrame(temp_results)
    temp_experiments = experiments[experiments.scenario==scenario]
    
    best = temp_results.min()
    scenario_regret = temp_results - best
    scenario_regret['policy'] = temp_experiments.policy.values    
    regret.append(scenario_regret)
    
    
regret = pd.concat(regret)
maxregret = regret.groupby('policy').max()

In [ ]:
limits = parcoords.get_limits(maxregret)
paraxes = parcoords.ParallelAxes(maxregret)
paraxes.plot(maxregret)
plt.show()